Note: In this competition the correct ensembling methods played a major role. First of all this competition contained two completly independed parts, which had as a consequence, that all teams had to train 2 different models for two different parts. The final score however was a combination of both. Therefore, keeping track of improvments independent from the other part requieres benchmarking. Moreover, the competition metric likes normalized results. Therefore, normalization is performed for every cell.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Imports

In [2]:
import numpy as np 
import pandas as pd 
from tqdm.notebook import tqdm
import os
import shutil

import gc

src = '/content/drive/MyDrive/Comps/Useful Stuff/kaggle.json'
dst = '/content/kaggle.json'
shutil.copyfile(src, dst)

#update kaggle api
!pip install kaggle

!mkdir ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Directories
dir_pred_multi  = '/content/drive/MyDrive/Comps/Multimodal_Single-Cell_Integration/Predictions/multi.parquet'
dir_sub         = '/content/drive/MyDrive/Comps/Multimodal_Single-Cell_Integration/Submissions'

# Files
dir_base_sub    = '/content/drive/MyDrive/Comps/Multimodal_Single-Cell_Integration/Submissions/submission.csv'
dir_cite_test   = '/content/drive/MyDrive/Comps/Multimodal_Single-Cell_Integration/Data/Cite/Test/test_cite_inputs.parquet'
eval_ids        = '/content/drive/MyDrive/Comps/Multimodal_Single-Cell_Integration/Data/evaluation_ids.csv'

dir_norm_cite   = '/content/drive/MyDrive/Comps/Multimodal_Single-Cell_Integration/Submissions/normalized_cite.parquet'
dir_norm_multi  = '/content/drive/MyDrive/Comps/Multimodal_Single-Cell_Integration/Submissions/normalized_multi.parquet'

### Functions ###

# Standardize
def std(x): return (x - np.mean(x)) / np.std(x)


# Standardize submission per cell_id
def gen_std_submission(path, cell_ids):

    df = pd.read_csv(path)
    df['cell_id'] = cell_ids 
    loop_list = df.groupby('cell_id', sort=False)
    vals = []

    for idx, g in tqdm(loop_list, desc=f'Standardizing {path}', miniters=1000):

        vals.append(std(g.target).values)
    
    return np.concatenate(vals)


# Ensemble Predictions
def gen_ensemble(technology):

    ensemble    = [0]* (58931360 if technology == 'multiome' else 6812820)
    submissions = multi_files    if technology == 'multiome' else cite_files
    folder      = dir_sub_multi  if technology == 'multiome' else dir_sub_cite
    cell_ids    = pd.read_csv(f'{folder}/cell_ids.csv')

    for path in tqdm(submissions.keys(), desc='Process submission'):
        
        weigth    =  submissions[path]
        pred_norm =  gen_std_submission(f'{folder}/{path}', cell_ids.cell_id) 
        ensemble  += pred_norm * weigth
        
    return ensemble

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Add Prediction

This section loads prediction obtained from the cite or multiome part and normalizes them by cell.

In [21]:
dir = '/content/drive/MyDrive/Comps/Multimodal_Single-Cell_Integration/Predictions/tianyu_svd.parquet'
dir = '/content/drive/MyDrive/Comps/Multimodal_Single-Cell_Integration/Predictions/tianyu_easy.parquet'
dir = '/content/drive/MyDrive/Comps/Multimodal_Single-Cell_Integration/Predictions/multi_catboost.parquet'
#dir = '/content/drive/MyDrive/Comps/Multimodal_Single-Cell_Integration/submission_onebestcite_xgboost_finalgroupfold.csv'

In [22]:
preds = pd.read_parquet(dir)

In [24]:
preds['mean_catboost'] = preds.mean(axis=1)

In [25]:
cell_ids           = pd.read_csv(f'{dir_sub}/cell_ids_multi.csv')
cell_ids['target'] = preds.mean_catboost.values

In [26]:
# Standardize submission per cell_id
loop_list          = cell_ids.groupby('cell_id', sort=False)
vals               = []

for idx, g in tqdm(loop_list, desc=f'Standardizing', miniters=1000):

    vals.append(std(g.target).values)

to_save_norm = np.concatenate(vals)

Standardizing:   0%|          | 0/16780 [00:00<?, ?it/s]

# Ensable 

This part loads different normalized predictions and combines them by weighting. 

In [3]:
cite  = pd.read_parquet(dir_norm_cite)
multi = pd.read_parquet(dir_norm_multi)

In [5]:
#         [  0,  0,  0,  0,  0,  .35, 1, .5, .1]
#         [  0,  0,  0,  0,  0,  .5, 1]
# best    [  0,  0,  0,  0,  0, .2, 1]

w_cite  = [  .1,  0,  0,  .15,  .25,  1, .2]
w_cite  = [  0,  0,  0,  0,  0,  .5, 1, 0, 0, .2]

In [6]:
list(zip(cite.columns, w_cite))

[('ambrosm', 0),
 ('sskknt', 0),
 ('xgb_lgbm_base', 0),
 ('juan5x5', 0),
 ('5/5_12', 0),
 ('tiuanyu_base', 0.35),
 ('ZJY27_base', 1),
 ('tiuanyu_imp', 0.5),
 ('ZJY27_imp', 0.1)]

In [49]:
#         [  0,   .01,  .01, 0,  .005,  .15, .15,  1, .15, .35, .15]
#         [.001,   0,   0,   0,  .005,  .2,  .1,  1]
#         [  0,   .1,  .125, 0,  .05,  .075, .15,  1]

#best   = [  0,    0,   0,   0,  .005,  .3,  .1,  1]
#second = [  0,    0,   0,   0,  .05,   .5,  .1,  1]

w_multi = [0, 0, 0, 0, .05, .2, .3, 1, 0, 0, 0, .15]

In [50]:
list(zip(multi.columns, w_multi))

[('pourchot', 0),
 ('juan5x5', 0),
 ('5/5_12', 0),
 ('x6_x8_models_mean', 0),
 ('mean_reg', 0.05),
 ('tianyu_60_to_80', 0.2),
 ('tianyu_base', 0.3),
 ('ZJY27_base', 1),
 ('ZJY27_imp', 0),
 ('tianyu_new_svd', 0),
 ('tianyu_easy_mean', 0),
 ('mean_catboost', 0.15)]

In [42]:
df_submit = pd.DataFrame(list(range(65744180)), columns = ['row_id'])
df_submit['target'] = 0

In [51]:
df_submit.loc[:6812819, 'target'] = cite.dot(w_cite).values
df_submit.loc[6812820:, 'target'] = multi.dot(w_multi).values

# Sumbit

In [52]:
df_submit.to_csv('/content/drive/MyDrive/Comps/Multimodal_Single-Cell_Integration/Submissions/ensemble.csv', index=False)

In [53]:
!kaggle competitions submit -c open-problems-multimodal -f /content/drive/MyDrive/Comps/Multimodal_Single-Cell_Integration/Submissions/ensemble.csv -m 'last try'

100% 1.74G/1.74G [00:30<00:00, 62.3MB/s]
Successfully submitted to Open Problems - Multimodal Single-Cell Integration